In [2]:
# coding: utf-8
from os import listdir, makedirs
from os.path import join, isfile, isdir, exists
import pandas as pd
import gc
from gensim.corpora import Dictionary, MmCorpus
from gensim.models import CoherenceModel, TfidfModel, LdaModel, LdaMulticore
from gensim.models.wrappers import LdaMallet
from gensim.models.hdpmodel import HdpModel, HdpTopicFormatter
from gensim.models.callbacks import CoherenceMetric, DiffMetric, PerplexityMetric, ConvergenceMetric
from itertools import chain, islice
from constants import (
    DATA_BASE, FULL_PATH, ETL_PATH, NLP_PATH, SMPL_PATH, POS, NOUN, PROPN, TOKEN, HASH, SENT_IDX, PUNCT
)
import logging
import json
import numpy as np
import visdom
vis = visdom.Visdom()

#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
#pd.options.display.max_rows = 2001

In [3]:
def docs_to_lists(token_series):
    return tuple(token_series.tolist())

def docs2corpora(documents, tfidf=False, stopwords=None, filter_below=5, filter_above=0.5,
                split=False, max_test_size_rel=0.1, max_test_size_abs=5000):

    print('building dictionary')
    dictionary = Dictionary(documents)
    dictionary.filter_extremes(no_below=filter_below, no_above=filter_above)

    # filter some noice (e.g. special characters)
    if stopwords:
        stopword_ids = [dictionary.token2id[token] for token in stopwords]
        dictionary.filter_tokens(bad_ids=stopword_ids, good_ids=None)
        
    length = len(documents)
    corpora = dict()
    if split:
        if length*max_test_size_rel < max_test_size_abs:
            split1 = int(length*(1-(2*max_test_size_rel)))
            split2 = int(length*(1-max_test_size_rel))
        else:
            split1 = length-(2*max_test_size_abs)
            split2 = length-max_test_size_abs
        training_texts = documents[:split1]
        holdout_texts = documents[split1:split2]
        test_texts = documents[split2:]
        print(
            f'split dataset. size of:',
            f'train_set={len(training_texts)},',
            f'val_set={len(holdout_texts)},',
            f'test_set={len(test_texts)},'
        )
        corpora['training_corpus'] = [dictionary.doc2bow(text) for text in training_texts]
        corpora['holdout_corpus'] = [dictionary.doc2bow(text) for text in holdout_texts]
        corpora['test_corpus'] = [dictionary.doc2bow(text) for text in test_texts]
    else:
        training_texts = documents
        corpora['training_corpus'] = [dictionary.doc2bow(text) for text in training_texts]
        corpora['holdout_corpus'], corpora['test_corpus'] = None, None

    if tfidf:
        for key, bow_corpus in corpora.items():
            tfidf_model = TfidfModel(bow_corpus)
            corpora[key] = tfidf_model[bow_corpus]
    return corpora, dictionary

In [4]:
datasets = {
    'E': 'Europarl',
    'FA': 'FAZ_combined',
    'FO': 'FOCUS_cleansed',
    'O': 'OnlineParticipation',
    'P': 'PoliticalSpeeches',
    'dewi': 'dewiki',
    'dewa': 'dewac',
}
goodids = {
    # filetered via some fixed rules and similarity measure to character distribution
    'dewac': join(ETL_PATH, 'dewac_good_ids.pickle'),
    'dewiki': join(ETL_PATH, 'dewiki_good_ids.pickle'),
    # the samples contain only a small subset of all articles
    # the reason for this is that the samples are roughly equal in size per category
    # 'FAZ_combined': join(ETL_PATH, 'FAZ_document_sample3.pickle'),
    # 'FOCUS_cleansed': join(ETL_PATH, 'FOCUS_document_sample3.pickle'),
}
bad_tokens = {
    'Europarl': [
        'E.', 'Kerr', 'The', 'la', 'ia', 'For', 'Ieke', 'the',
    ],
    'FAZ_combined': [
        'S.', 'j.reinecke@faz.de', 'B.',
    ],
    'FOCUS_cleansed': [],
    'OnlineParticipation': [
        'Re', '@#1', '@#2', '@#3', '@#4', '@#5', '@#6', '@#7', '@#8', '@#9', '@#1.1', 'Für', 'Muss',
        'etc', 'sorry', 'Ggf', 'u.a.', 'z.B.', 'B.', 'stimmt', ';-)', 'lieber', 'o.', 'Ja', 'Desweiteren',
    ],
    'PoliticalSpeeches': [],
    'dewiki': [],
    'dewac': [],
}
all_bad_tokens = set(chain(*bad_tokens.values()))

In [5]:
dataset = datasets['dewi']
print('dataset:', dataset)
print('-' * 5)

sub_dir = 'dewiki' if dataset.startswith('dewi') else 'wiki_phrases'
dir_path = join(SMPL_PATH, sub_dir)
files = sorted([f for f in listdir(dir_path) if f.startswith(dataset)])

keepids = None
if dataset in goodids:
    keepids = pd.read_pickle(goodids[dataset])

nb_words = 0
documents = []
for name in files[:]:
    gc.collect()
    full_path = join(dir_path, name)
    if not isfile(full_path):
        continue

    print('reading', name)
    df = pd.read_pickle(join(dir_path, name))
    print('    initial number of words:', len(df))
    if keepids is not None:
        # some datasets have already been filtered so you may not see a difference in any case
        df = df[df.hash.isin(keepids.index)]

    # fixing bad POS tagging
    mask = df.token.isin(list('[]<>/–%'))
    df.loc[mask, POS] = PUNCT

    # using only certain POS tags
    reduction_pos_set = {NOUN, PROPN, 'NER', 'NPHRASE'}
    df = df[df.POS.isin(reduction_pos_set)]
    df[TOKEN] = df[TOKEN].map(lambda x: x.strip('-/'))
    df = df[df.token.str.len() > 1]
    df = df[~df.token.isin(all_bad_tokens)]
    nb_words += len(df)
    print('    remaining number of words:', len(df))
    # groupby sorts the documents by hash-id which is equal to shuffeling the dataset before building the model
    df = df.groupby([HASH])[TOKEN].agg(docs_to_lists)
    print('    number of documents:', len(df))
    documents += df.values.tolist()

nb_docs = len(documents)
print('-' * 5)
print('total number of documents:', nb_docs)
print('total number of words:', nb_words)
stats = dict(dataset=dataset, pos_set=sorted(reduction_pos_set), nb_docs=nb_docs, nb_words=nb_words)
del keepids, files
gc.collect();

dataset: dewiki
-----
reading dewiki_new_0_49999__10000_simple.pickle
    initial number of words: 19181198
    remaining number of words: 4996598
    number of documents: 8810
reading dewiki_new_0_49999__20000_simple.pickle
    initial number of words: 14043977
    remaining number of words: 3682812
    number of documents: 9094
reading dewiki_new_0_49999__30000_simple.pickle
    initial number of words: 12613028
    remaining number of words: 3314546
    number of documents: 9645
reading dewiki_new_0_49999__40000_simple.pickle
    initial number of words: 11172466
    remaining number of words: 2940768
    number of documents: 9664
reading dewiki_new_0_49999__46920_simple.pickle
    initial number of words: 7110900
    remaining number of words: 1867987
    number of documents: 6843
reading dewiki_new_1000000_1049999__10000_simple.pickle
    initial number of words: 3675645
    remaining number of words: 1018612
    number of documents: 9924
reading dewiki_new_1000000_1049999__20000_

    initial number of words: 3333051
    remaining number of words: 919204
    number of documents: 9917
reading dewiki_new_1400000_1449999__30000_simple.pickle
    initial number of words: 3197052
    remaining number of words: 882916
    number of documents: 9904
reading dewiki_new_1400000_1449999__40000_simple.pickle
    initial number of words: 3050597
    remaining number of words: 846235
    number of documents: 9917
reading dewiki_new_1400000_1449999__41167_simple.pickle
    initial number of words: 386116
    remaining number of words: 106317
    number of documents: 1149
reading dewiki_new_1450000_1499999__10000_simple.pickle
    initial number of words: 3246915
    remaining number of words: 901245
    number of documents: 9938
reading dewiki_new_1450000_1499999__20000_simple.pickle
    initial number of words: 3158872
    remaining number of words: 877946
    number of documents: 9931
reading dewiki_new_1450000_1499999__30000_simple.pickle
    initial number of words: 321702

    initial number of words: 3054448
    remaining number of words: 851483
    number of documents: 9957
reading dewiki_new_1850000_1899999__40000_simple.pickle
    initial number of words: 3259931
    remaining number of words: 906862
    number of documents: 9933
reading dewiki_new_1850000_1899999__43564_simple.pickle
    initial number of words: 1156046
    remaining number of words: 320352
    number of documents: 3550
reading dewiki_new_1900000_1949999__10000_simple.pickle
    initial number of words: 3158034
    remaining number of words: 880240
    number of documents: 9936
reading dewiki_new_1900000_1949999__20000_simple.pickle
    initial number of words: 3157701
    remaining number of words: 887608
    number of documents: 9939
reading dewiki_new_1900000_1949999__30000_simple.pickle
    initial number of words: 3203903
    remaining number of words: 893180
    number of documents: 9953
reading dewiki_new_1900000_1949999__40000_simple.pickle
    initial number of words: 32700

    remaining number of words: 1136972
    number of documents: 8920
reading dewiki_new_350000_399999__30000_simple.pickle
    initial number of words: 4007088
    remaining number of words: 1067376
    number of documents: 8637
reading dewiki_new_350000_399999__40000_simple.pickle
    initial number of words: 4351563
    remaining number of words: 1153654
    number of documents: 8920
reading dewiki_new_350000_399999__50000_simple.pickle
    initial number of words: 3982088
    remaining number of words: 1063083
    number of documents: 8785
reading dewiki_new_400000_449999__10000_simple.pickle
    initial number of words: 3488717
    remaining number of words: 924003
    number of documents: 8269
reading dewiki_new_400000_449999__20000_simple.pickle
    initial number of words: 3514507
    remaining number of words: 932951
    number of documents: 8409
reading dewiki_new_400000_449999__30000_simple.pickle
    initial number of words: 3877315
    remaining number of words: 1033919
   

    initial number of words: 3758589
    remaining number of words: 1034537
    number of documents: 9944
reading dewiki_new_800000_849999__43075_simple.pickle
    initial number of words: 1096908
    remaining number of words: 304057
    number of documents: 3055
reading dewiki_new_850000_899999__10000_simple.pickle
    initial number of words: 3656430
    remaining number of words: 1006013
    number of documents: 9946
reading dewiki_new_850000_899999__20000_simple.pickle
    initial number of words: 3600400
    remaining number of words: 992441
    number of documents: 9947
reading dewiki_new_850000_899999__30000_simple.pickle
    initial number of words: 3663934
    remaining number of words: 1010259
    number of documents: 9912
reading dewiki_new_850000_899999__40000_simple.pickle
    initial number of words: 3525957
    remaining number of words: 970886
    number of documents: 9885
reading dewiki_new_850000_899999__42501_simple.pickle
    initial number of words: 841378
    rem

In [6]:
# save full dataset as bow/tfidf corpus in Matrix Market format, including dictionary
use_tfidf = False
corpus_type = 'tfidf' if use_tfidf else 'bow'
split = False
split_type = 'trainset' if split else 'fullset'
corpora, dictionary = docs2corpora(
    documents, tfidf=use_tfidf,
    filter_below=5, filter_above=0.5,
    split=split,
)
corpus = corpora['training_corpus']
file_name = f'{dataset}_{split_type}_nouns_{corpus_type}'
corpus_path = join(ETL_PATH, 'LDAmodel', file_name + '.mm')
print(f'saving {corpus_path}')
MmCorpus.serialize(corpus_path, corpus)
dict_path = join(ETL_PATH, 'LDAmodel', file_name + '.dict')
print(f'saving {dict_path}')
dictionary.save(dict_path)
doc_path = join(ETL_PATH, 'LDAmodel', file_name.rstrip(f'{corpus_type}') + 'texts.json')
print(f'saving {doc_path}')
with open(doc_path, 'w') as fp:
    json.dump(documents, fp, ensure_ascii=False)

building dictionary
saving ../data/preprocessed/LDAmodel/dewiki_fullset_nouns_bow.mm
saving ../data/preprocessed/LDAmodel/dewiki_fullset_nouns_bow.dict
saving ../data/preprocessed/LDAmodel/dewiki_fullset_nouns_texts.json


In [ ]:
# initialize training and test data
corpora, dictionary = docs2corpora(
    documents, tfidf=False,
    #stopwords=bad_tokens[dataset],  # stopword removale has been moved to the pandas preprocessing pipeline
    filter_below=5, filter_above=0.5,
    split=True,
)
training_corpus = corpora['training_corpus']
holdout_corpus = corpora['holdout_corpus']
test_corpus = corpora['test_corpus']

def init_callbacks(viz_env=None, title_suffix=''):
    # define perplexity callback for hold_out and test corpus
    pl_holdout = PerplexityMetric(
        corpus=holdout_corpus, logger="visdom", viz_env=viz_env, title="Perplexity (hold_out)"+title_suffix
    )
    pl_test = PerplexityMetric(
        corpus=test_corpus, logger="visdom", viz_env=viz_env, title="Perplexity (test)"+title_suffix
    )
    # define other remaining metrics available
    ch_umass = CoherenceMetric(
        corpus=training_corpus, coherence="u_mass", topn=10, logger="visdom", viz_env=viz_env, title="Coherence (u_mass)"+title_suffix
    )
    ch_cv = CoherenceMetric(
        corpus=training_corpus, texts=documents, coherence="c_v", topn=10, logger="visdom", viz_env=viz_env, title="Coherence (c_v)"+title_suffix
    )
    diff_kl = DiffMetric(
        distance="kullback_leibler", logger="visdom", viz_env=viz_env, title="Diff (kullback_leibler)"+title_suffix
    )
    convergence_kl = ConvergenceMetric(
        distance="jaccard", logger="visdom", viz_env=viz_env, title="Convergence (jaccard)"+title_suffix
    )
    return [pl_holdout, pl_test, ch_umass, ch_cv, diff_kl, convergence_kl]

In [ ]:
def get_parameterset(corpus, dictionary, callbacks=None, nbtopics=100, parametrization='a42', eval_every=None):
    print(f'building LDA model "{parametrization}" with {nbtopics} number of topics')
    default = dict(random_state=42, corpus=corpus, id2word=dictionary, num_topics=nbtopics, 
                   eval_every=eval_every, callbacks=callbacks, chunksize=20_000)
    ldamodels = {
        'a42': dict(passes=10),
        'b42': dict(passes=10, iterations=200),
        'c42': dict(passes=10, iterations=1_000),
        'd42': dict(passes=10, iterations=200, alpha=0.1, eta=0.01),
        'e42': dict(passes=20, iterations=200, alpha='auto', eta='auto'),
    }
    for key, dic in ldamodels.items():
        dic.update(default)
    return ldamodels[parametrization]

Install Visdom via ```pip install visdom``` and run ```python -m visdom.server``` to start the server.
You will be able to view online log stats @ http://localhost:8097/

Attention: gensim is currently not fully compatible with visdom. See https://github.com/RaRe-Technologies/gensim/issues/2155 for details.

To fix this issue change in gensim.models.callbacks.Callback.on_epoch_end the following line
```
self.viz.updateTrace(
    Y=np.array([value]), X=np.array([epoch]), env=metric.viz_env, win=self.windows[i]
)
```
to
```
self.viz.line(
    Y=np.array([value]), X=np.array([epoch]), env=metric.viz_env, win=self.windows[i], update='append'
)
```
You might need to re-import gensim (and probably even restart this notebook).

In [ ]:
save = False

In [ ]:
params_list = ['a42', 'b42', 'c42', 'd42', 'e42']
implementations = [
    ('LDAmodel', LdaModel),
    ('LDAmulticore', LdaMulticore),
    ('LDAmallet', LdaMallet),
]
choice = 2
model_name = implementations[choice][0]
UsedModel = implementations[choice][1]
metrics = []
topn = 20
# params = params_list[3]
env_id = f"{dataset}-{model_name}"
for params in params_list[:1]:
    for nbtopics in [10, 25, 50, 100][:1]:  # range(10, 101, 10):
        # Choose α from [0.05, 0.1, 0.5, 1, 5, 10]
        # Choose β from [0.05, 0.1, 0.5, 1, 5, 10]
        callbacks = init_callbacks(
            viz_env=env_id,
            title_suffix=f", {params}, {nbtopics}tpx"
        )
        kwargs = get_parameterset(
            training_corpus,
            dictionary,
            callbacks=callbacks,
            nbtopics=nbtopics,
            parametrization=params
        )
        if 'multicore' in model_name:
            kwargs['workers'] = 3
            kwargs.pop('callbacks', None)
        elif 'mallet' in model_name:
            kwargs['workers'] = 3
            kwargs['mallet_path'] = join(DATA_BASE, 'Mallet', 'bin', 'mallet')
            kwargs.pop('passes', None)
            kwargs.pop('random_state', None)
            kwargs.pop('eval_every', None)
            kwargs.pop('callbacks', None)
            kwargs.pop('chunksize', None)

        print('running {model_name}')
        ldamodel = UsedModel(**kwargs)

        topics = [
            [dataset] +
            [dictionary[term[0]] for term in ldamodel.get_topic_terms(i, topn=topn)]
            for i in range(nbtopics)
        ]
        df_lda = pd.DataFrame(topics, columns=['dataset'] + ['term' + str(i) for i in range(topn)])

        current_metrics = ldamodel.metrics
        # print(current_metrics)
        metrics.append(('env_id', current_metrics))

        if save:
            out_dir = join(ETL_PATH, f'{model_name}/{params}')
            if not exists(out_dir):
                makedirs(out_dir)
            out = join(out_dir, f'{dataset}_{model_name}_{params}_{nbtopics}')
            print('saving to', out)
            df_lda.to_csv(out + '.csv')
            ldamodel.save(out)
            with open(out + '_stats.json', 'w') as fp:
                json.dump(stats, fp)
            with open(out + '_metrics.json', 'w') as fp:
                serializable_metrics = {}
                for k, v in current_metrics.items():
                    if isinstance(v[0], np.ndarray):
                        serializable_metrics[k] = [x.tolist() for x in v]
                    else:
                        serializable_metrics[k] = [float(x) for x in v]
                json.dump(serializable_metrics, fp)
            vis.save([env_id])


-----

### Evaluation

In [ ]:
import pyLDAvis as ldavis
import pyLDAvis.gensim

ldavis.enable_notebook()
prepared_data = ldavis.gensim.prepare(ldamodel, training_corpus, dictionary)
prepared_data